In [15]:
!pip install kagglehub

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 KB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 KB 7.4 MB/s eta 0:00:00


In [16]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("umairaslam/credit-card")

print("Path to dataset files:", path)

/home/violet/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 274k/274k [00:01<00:00, 253kB/s]

Extracting files...
Path to dataset files: /home/violet/.cache/kagglehub/datasets/umairaslam/credit-card/versions/1


In [28]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [22]:
df = pd.read_csv('/home/violet/.cache/kagglehub/datasets/umairaslam/credit-card/versions/1/credit_card.csv')

df.head()
df.info()
df.isnull().sum()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CUST_ID                 8950 non-null   object 
 1   BALANCE                 8950 non-null   float64
 2   PURCHASES               8950 non-null   float64
 3   INSTALLMENTS_PURCHASES  8950 non-null   float64
 4   CASH_ADVANCE            8950 non-null   float64
 5   CREDIT_LIMIT            8950 non-null   float64
 6   PAYMENTS                8950 non-null   float64
 7   MINIMUM_PAYMENTS        8950 non-null   float64
 8   TENURE                  8950 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 629.4+ KB


,BALANCE,PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,TENURE
count,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000
mean,1564.474828,1003.204834,411.067645,978.871112,4494.282473,1733.143852,844.906767,11.517318
std,2081.531879,2136.634782,904.338115,2097.163877,3638.646702,2895.063757,2332.792322,1.338331
min,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.019163,6.000000
25%,128.281915,39.635000,0.000000,0.000000,1600.000000,383.276166,170.857654,12.000000
50%,873.385231,361.280000,89.000000,0.000000,3000.000000,856.901546,312.343947,12.000000
75%,2054.140036,1110.130000,468.637500,1113.821139,6500.000000,1901.134317,788.713501,12.000000
max,19043.138560,49039.570000,22500.000000,47137.211760,30000.000000,50721.483360,76406.207520,12.000000


In [24]:
df = df.drop(columns=['CUST_ID'])

In [25]:
assert df.isnull().sum().sum() ==0

In [26]:
scaler = RobustScaler()

In [38]:
silhouette_scores = {}
inertias = {}

for k in range(2,11):
    pipeline = Pipeline(steps=[('scaler',scaler),
                               ('kmeans',KMeans(
                                   n_clusters=k,
                                   random_state=42,
                                   n_init=20
                                   ))
                                   ])
    labels = pipeline.fit_predict(df)

    silhouette_scores[k] = silhouette_score(
        pipeline.named_steps["scaler"].transform(df),
        labels
    )
    inertias[k] = pipeline.named_steps['kmeans'].inertia_

In [39]:
best_k = max(silhouette_scores,key=silhouette_scores.get)

print("Silhouette Scores:", silhouette_scores)
print("Best number of clusters:",best_k)

Silhouette Scores: {2: 0.8630216657463016, 3: 0.5885215412141493, 4: 0.5170362967145163, 5: 0.48970938149660526, 6: 0.46470952822651856, 7: 0.43622704989040306, 8: 0.4363785493243665, 9: 0.434987381620087, 10: 0.39077396885052}
Best number of clusters: 2


In [43]:
final_pipeline = Pipeline(steps=[
    ("scaler", RobustScaler()),
    ("kmeans", KMeans(
        n_clusters=best_k,
        random_state=42,
        n_init=30
    ))
])

In [45]:
cluster_labels = final_pipeline.fit_predict(df)

df['Cluster'] = cluster_labels

cluster_profiles = df.groupby('Cluster').mean()

print(cluster_profiles)

In [46]:
pca = PCA(n_components=2,random_state=42)

X_scaled = final_pipeline.named_steps['scaler'].transform(df.drop(columns=['Cluster']))
X_pca = pca.fit_transform(X_scaled)

df["PCA1"] = X_pca[:,0]
df["PCA2"] = X_pca[:,1]


In [52]:
for cluster_id in sorted(df["Cluster"].unique()):
    print(f"\n--- Cluster {cluster_id} Summary ---")
    print(cluster_profiles.loc[cluster_id])


--- Cluster 0 Summary ---


NameError: name 'cluster_profiles' is not defined

In [ ]:
df.to_csv("credit_card_customers_segmented.csv", index=False)

print("\n✅ Customer segmentation complete.")

In [53]:
import pickle

with open("credit_card_kmeans_pipeline.pkl", "wb") as f:
    pickle.dump(final_pipeline, f)


In [55]:
from joblib import dump, load

dump(final_pipeline, "credit_card_kmeans_pipeline.joblib")
loaded_pipeline = load("credit_card_kmeans_pipeline.joblib")


In [58]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [("float_input", FloatTensorType([None, 8]))]
onnx_model = convert_sklearn(final_pipeline, initial_types=initial_type)

with open("credit_card_kmeans.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())


In [60]:
from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline

pmml_pipeline = PMMLPipeline(steps=final_pipeline.steps)
pmml_pipeline.fit(df.drop("Cluster", axis=1))

sklearn2pmml(pmml_pipeline, "credit_card_model.pmml")


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 4.4 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 5.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 4.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.3/262.3 KB 4.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 5.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 KB 1

In [ ]:
import mlflow
import mlflow.sklearn

mlflow.sklearn.log_model(final_pipeline, "kmeans_model")
